### import

In [73]:
import random 
import pandas as pd 
import numpy as np
import os 
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A 
import albumentations
import albumentations.pytorch
from albumentations.pytorch.transforms import ToTensorV2 
import torchvision.models as models
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler

In [74]:
train_df = pd.read_csv('C:/Users/moond/open/Data/train2.csv',encoding='utf-8')
train_df.head(5)

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식


In [75]:
train_df['img']=".jpg" 
train_df.head(5)

,id,img_path,overview,cat1,cat2,cat3,img
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,.jpg
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,.jpg
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,.jpg
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,.jpg
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,.jpg


In [76]:
train_df['img_name']= train_df['id']+train_df['img']
train_df.head(5)

,id,img_path,overview,cat1,cat2,cat3,img,img_name
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,.jpg,TRAIN_00000.jpg
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,.jpg,TRAIN_00001.jpg
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,.jpg,TRAIN_00002.jpg
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,.jpg,TRAIN_00003.jpg
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,.jpg,TRAIN_00004.jpg


In [95]:
train_df["cat3"] = train_df["cat3"].str.replace(pat=r'[^\w]', repl=',', regex=True)

In [96]:
train_df["cat3"]

0            항구,포구
1               골프
2               한식
3               한식
4               한식
           ...    
16981           한식
16982           모텔
16983           모텔
16984    야영장,오토캠핑장
16985           사찰
Name: cat3, Length: 16986, dtype: object

In [104]:
train_df.head(5)

,id,img_path,overview,cat1,cat2,cat3,img,img_name
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,"항구,포구",.jpg,TRAIN_00000.jpg
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,.jpg,TRAIN_00001.jpg
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,.jpg,TRAIN_00002.jpg
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,.jpg,TRAIN_00003.jpg
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,.jpg,TRAIN_00004.jpg


In [105]:
print(len(train_df["cat3"].unique())) # type = numpy.array

128


In [106]:
label_list = train_df["cat3"].unique().tolist()

In [107]:
len(label_list)  # type = list

128

#### 1. cat3 종류별 폴더 만들기

In [108]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 
for i in range(len(label_list)):  # cat3 개수 만큼 
    createFolder(f'./image/train/{label_list[i]}')  # 폴더를 생성 

#### 2. 이미지 파일을 cat3에 맞게 각 폴더로 옮기기

In [109]:
train_folder = os.listdir('./image/train/')
len(train_folder) 

17114

In [110]:
import shutil

In [112]:
for i in range(len(train_folder)-128):  
    
    if train_folder[i][-3:] == "jpg":  
        label = train_df.loc[train_df["img_name"] == f"{train_folder[i]}"]["cat3"].item() 
        file_source = f'C:/Users/moond/open/Capstone/image/train/{train_folder[i]}'  
        file_destination = f'C:/Users/moond/open/Capstone/image/train/{label}/' 
        shutil.move(file_source, file_destination)  

#### 3. 이미지 데이터 부르기

In [113]:
train_path = 'C:/Users/moond/open/Capstone/image/train'
test_path = 'C:/Users/moond/open/Capstone/image/test'

In [114]:
def read_image(train_path):
    img_array=cv2.imread(train_path)
    return cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB) # BGR을 RGB로 바꿔어줌

###### CV2를 이용해 이미지 불러오기

#### 4. 이미지 normalization

In [115]:
import numpy as np
import torchvision.transforms as transforms
from torchvision import datasets
import torch
import torchvision
from torchvision import transforms 
from torch.utils.data import DataLoader 
import os
import glob
from torchvision.datasets import ImageFolder

In [116]:
resize_transform = transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor()
])

In [118]:
resize_train = ImageFolder(root=train_path, transform=resize_transform)

In [119]:
resize_train[0][0]

tensor([[[0.5569, 0.5529, 0.5451,  ..., 0.5255, 0.2314, 0.2235],
         [0.5647, 0.5569, 0.5529,  ..., 0.5137, 0.2275, 0.2196],
         [0.5451, 0.5451, 0.5490,  ..., 0.4980, 0.2275, 0.2196],
         ...,
         [0.7686, 0.7686, 0.7686,  ..., 0.7412, 0.7294, 0.6784],
         [0.7725, 0.7843, 0.7608,  ..., 0.7961, 0.7843, 0.7686],
         [0.7373, 0.7608, 0.7686,  ..., 0.7882, 0.7804, 0.7608]],

        [[0.7529, 0.7490, 0.7412,  ..., 0.5961, 0.2471, 0.2314],
         [0.7569, 0.7529, 0.7490,  ..., 0.5765, 0.2431, 0.2275],
         [0.7373, 0.7373, 0.7412,  ..., 0.5490, 0.2431, 0.2314],
         ...,
         [0.7333, 0.7373, 0.7412,  ..., 0.7137, 0.7020, 0.6510],
         [0.7333, 0.7490, 0.7333,  ..., 0.7490, 0.7412, 0.7255],
         [0.6980, 0.7255, 0.7412,  ..., 0.7412, 0.7333, 0.7137]],

        [[0.9843, 0.9804, 0.9725,  ..., 0.6980, 0.3294, 0.3137],
         [0.9882, 0.9843, 0.9765,  ..., 0.6745, 0.3216, 0.3059],
         [0.9647, 0.9647, 0.9686,  ..., 0.6471, 0.3176, 0.

In [120]:
resize_train[0][0].shape

torch.Size([3, 150, 150])

In [121]:
import numpy as np
np.mean(resize_train[0][0].numpy(),axis=(1,2))

array([0.47081935, 0.4978224 , 0.5266998 ], dtype=float32)

In [122]:
def get_mean_std(dataset):
  meanRGB = [np.mean(image.numpy(), axis=(1,2)) for image,_ in dataset]
  stdRGB = [np.std(image.numpy(), axis=(1,2)) for image,_ in dataset]

  meanR = np.mean([m[0] for m in meanRGB])
  meanG = np.mean([m[1] for m in meanRGB])
  meanB = np.mean([m[2] for m in meanRGB])

  stdR = np.mean([s[0] for s in stdRGB])
  stdG = np.mean([s[1] for s in stdRGB])
  stdB = np.mean([s[2] for s in stdRGB])

  print(meanR, meanG, meanB)
  print(stdR, stdG, stdB)

In [123]:
get_mean_std(resize_train)

0.49876195 0.49929163 0.47256655
0.23816794 0.23680994 0.2649413


#### 5. data augmentation 및 data split

In [129]:
data_dir = "./"

def image_datasets(batch_size):
    
    train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 좌우반전 
    transforms.RandomVerticalFlip(),  # 상하반전 
    transforms.Resize((150, 150)),   
    transforms.ToTensor(), 
    transforms.Normalize([0.49876195, 0.49929163, 0.47256655], [0.2381674, 0.23680994, 0.2649413])
])

    train_data = datasets.ImageFolder(train_path , train_transform)

    valid_size = 0.2 

    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=train_sampler,
                                               num_workers=0)


    valid_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=valid_sampler,
                                               num_workers=0)

    return train_data, train_loader, valid_loader

In [130]:
train_data, train_loader, valid_loader = image_datasets(batch_size=4)


print('train 데이터셋 크기:', len(train_data))

class_names = train_data.classes
print('클래스:', class_names)

train 데이터셋 크기: 16986
클래스: ['5일장', 'ATV', 'MTB', '강', '게스트하우스', '계곡', '고궁', '고택', '골프', '공연장', '공예,공방', '공원', '관광단지', '국립공원', '군립공원', '기념관', '기념탑,기념비,전망대', '기암괴석', '기타', '기타행사', '농,산,어촌,체험', '다리,대교', '대중콘서트', '대형서점', '도립공원', '도서관', '동굴', '동상', '등대', '래프팅', '면세점', '모텔', '문', '문화관광축제', '문화원', '문화전수시설', '뮤지컬', '미술관,화랑', '민물낚시', '민박', '민속마을', '바,까페', '바다낚시', '박람회', '박물관', '발전소', '백화점', '번지점프', '복합,레포츠', '분수', '빙벽등반', '사격장', '사찰', '산', '상설시장', '생가', '서비스드레지던스', '서양식', '섬', '성', '수련시설', '수목원', '수상레포츠', '수영', '스노쿨링,스킨스쿠버다이빙', '스카이다이빙', '스케이트', '스키,보드,,렌탈샵', '스키,스노보드', '승마', '식음료', '썰매장', '안보관광', '야영장,오토캠핑장', '약수터', '연극', '영화관', '온천,욕장,스파', '외국문화원', '요트', '윈드서핑,제트스키', '유람선,잠수함관광', '유명건물', '유스호스텔', '유원지', '유적지,사적지', '이색거리', '이색찜질방', '이색체험', '인라인,실내,인라인,포함,', '일반축제', '일식', '자동차경주', '자연생태관광지', '자연휴양림', '자전거하이킹', '전문상가', '전시관', '전통공연', '종교성지', '중식', '채식전문점', '카약,카누', '카지노', '카트', '컨벤션', '컨벤션센터', '콘도미니엄', '클래식음악회', '클럽', '터널', '테마공원', '트래킹', '특산물판매점', '패밀리레스토랑', '펜션', '폭포', '학교', '한식', '한옥스테이', '항구,

#### 6. cnn model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        #1번째 층
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, , kernel_size= , stride= 1, padding= 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size= , stride= 2)
        )
        
        #2번째 층
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(, , kernel_size= , stride= 1, padding= 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size= , stride= 2)
        )
        
        #3번째 층
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(, , kernel_size= , stride= 1, padding= 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size= , stride= 2)
        )
        
        self.fc1 = torch.nn.Linear(, , bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        
            self.fc2 = torch.nn.Linear(, , bias=True)
            torch.nn.init.xavier_uniform_(self.fc2.weight)
            
        def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)  
        out = self.layer4(out)
        out = self.fc2(out)
        return out

### test